In [1]:
import model

/home/cs/miniconda3/envs/tensorflow1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cs/miniconda3/envs/tensorflow1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cs/miniconda3/envs/tensorflow1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cs/miniconda3/envs/tensorflo

In [2]:
from __future__ import division
import matplotlib.pyplot as plt
%matplotlib inline
#from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.activations import softmax, relu, sigmoid
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import cv2
from random import shuffle
import random
import glob
import numpy as np
from skimage.transform import resize
#from PIL import Image
import os
from model import *
from keras.utils import to_categorical
from sklearn.metrics import precision_recall_fscore_support

from collections import defaultdict, Counter
import itertools
from tqdm import tqdm
import time
import shutil
from datetime import datetime
import json

In [3]:
from inception_resnet_v2_2_1_1 import *

In [4]:
from dataGenerator import generate_experiment_data

In [5]:
num_classes = 2

In [6]:

def batch_generator(data_X, data_y, batch_size):
    indexes = np.array(range(len(data_y)))
    n = len(indexes)
    while True:
        batch_start = 0
        batch_end = batch_size
        np.random.shuffle(indexes)
        while batch_start < n:
            index = []
            batch_y = []
            y = []
            index = indexes[batch_start:batch_end]
            batch_x = np.array([data_X[i] for i in index])
            batch_y = np.array([data_y[i] for i in index])
            yield batch_x, batch_y
            batch_start += batch_size   
            batch_end += batch_size
            if (batch_end>len(data_y)):
                batch_end = len(data_y)


In [7]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [8]:
def my_confusion_matrix(y_test, y_pred, classes):
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_pred)

    np.set_printoptions(precision=2)
    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=classes,title='Confusion matrix, without normalization')

    # Plot normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=classes, normalize=True,title='Normalized confusion matrix')

    plt.show()

    return cnf_matrix

In [9]:
def conduct_experiment(exp, nb_epoch=200):
    Train_images = np.array([cv2.imread(image_name) for image_name in exp.trainX])
    Test_images = np.array([cv2.imread(image_name) for image_name in exp.testX])
    
    model_name = f'InceptionResNet_Slice{exp.slice}_Testing_{exp.testMonkey}'
    Train_X = Train_images
    Train_y = to_categorical([0 if i in ['W0', 'W2'] else 1 for i in exp.trainY], num_classes)

    Test_X = Test_images
    Test_y = to_categorical([0 if i in ['W0', 'W2'] else 1 for i in exp.testY], num_classes)

    X_train, X_val, y_train, y_val = train_test_split(Train_X, Train_y, test_size=0.2)
    
    batch_size = 10
    train_gen = batch_generator(X_train, y_train, batch_size)
    val_gen = batch_generator(X_val, y_val, batch_size)
    test_gen = batch_generator(Test_X, Test_y, batch_size)
    
    input_shape=(256,256,3)
    model =MyInceptionResNetV2(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    counts = exp.trainY.value_counts()
    class_weights = {
        0: counts[0]/counts[0],
        1: counts[0]/counts[1],}
    
    checkpoint = ModelCheckpoint('WEIGHTS_LeftlLung_Including_W16_W20/'+model_name+'.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto')
    tensorboard = TensorBoard(log_dir='EVENTS/', batch_size=batch_size, write_graph=True, write_images=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.00001)
    earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=2, mode='auto')
    cvslogger = keras.callbacks.CSVLogger('WEIGHTS_LeftlLung_Including_W16_W20/'+model_name+'.csv', separator=',', append=True)

    callbacks = [checkpoint, reduce_lr, earlystop, cvslogger, tensorboard]
    train_steps = int(len(y_train)//batch_size)
    val_steps = int(len(Test_y)//batch_size)
    start_time = time.time()
    history = model.fit_generator(train_gen, train_steps, epochs=nb_epoch, verbose=1, 
                        max_queue_size=2, validation_data=val_gen, 
                        validation_steps=val_steps, shuffle=True,
                        workers=1, use_multiprocessing= False, 
                        initial_epoch=0, callbacks=callbacks, class_weight=class_weights)
    print("Seconds: ", time.time() - start_time)
    
    y_pred = model.predict(Test_X)
    preds=y_pred
    pres_classes = []
    for p in preds.argmax(axis=1):
        pres_classes.append(class_labels[p])
    
    truth_classes = []
    for p in Test_y.argmax(axis=1):
        truth_classes.append(class_labels[p])
    
    p, r, _, _ = precision_recall_fscore_support(truth_classes, pres_classes)

    p = p.mean()
    r = r.mean()

    return {
        'slice': exp.slice,
        'monkey': exp.testMonkey,
        'model_name': model_name,
        'prediction': pres_classes,
        'truth': truth_classes,
        'actual_weeks': list(exp.testY),
        'precision': p,
        'recall': r,
        'f1': (2 * p * r) / (p + r)
    }


In [10]:
classes = class_labels = {0: 'notb', 1: 'tb'}

In [11]:
experiment = 'LeftlLung_Including_W16_W20'

In [12]:
outputCsv = f'{experiment}.csv'
outputPkl = f'{experiment}.pkl'

results = []

for exp in generate_experiment_data(experiment, slices=range(4, 10)):
    results.append(conduct_experiment(exp, nb_epoch=200))

df = pd.DataFrame(results)

Epoch 1/200
156/156 [==============================] - 50s 318ms/step - loss: 0.6907 - acc: 0.7429 - val_loss: 0.9422 - val_acc: 0.6333

Epoch 00001: val_loss improved from inf to 0.94221, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice4_Testing_MMMU36727.h5
Epoch 2/200
156/156 [==============================] - 11s 69ms/step - loss: 0.4914 - acc: 0.8032 - val_loss: 0.8496 - val_acc: 0.5778

Epoch 00002: val_loss improved from 0.94221 to 0.84960, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice4_Testing_MMMU36727.h5
Epoch 3/200
156/156 [==============================] - 11s 69ms/step - loss: 0.4428 - acc: 0.8308 - val_loss: 3.9068 - val_acc: 0.7556

Epoch 00003: val_loss did not improve
Epoch 4/200
156/156 [==============================] - 11s 70ms/step - loss: 0.2960 - acc: 0.8917 - val_loss: 1.3284 - val_acc: 0.6056

Epoch 00004: val_loss did not improve
Epoch 5/200
156/156 [==============================] - 11s 70ms/step - loss: 0.

/home/cs/miniconda3/envs/tensorflow1.8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/200
145/145 [==============================] - 13s 92ms/step - loss: 0.5721 - acc: 0.7683 - val_loss: 1.1476 - val_acc: 0.8258

Epoch 00001: val_loss improved from inf to 1.14757, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice4_Testing_MMMU36365.h5
Epoch 2/200
145/145 [==============================] - 11s 74ms/step - loss: 0.4319 - acc: 0.8212 - val_loss: 0.2900 - val_acc: 0.8885

Epoch 00002: val_loss improved from 1.14757 to 0.28996, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice4_Testing_MMMU36365.h5
Epoch 3/200
145/145 [==============================] - 10s 70ms/step - loss: 0.3664 - acc: 0.8620 - val_loss: 0.3110 - val_acc: 0.8656

Epoch 00003: val_loss did not improve
Epoch 4/200
145/145 [==============================] - 10s 70ms/step - loss: 0.3326 - acc: 0.8827 - val_loss: 2.0889 - val_acc: 0.8131

Epoch 00004: val_loss did not improve
Epoch 5/200
145/145 [==============================] - 10s 70ms/step - loss: 0.2

Epoch 16/200
135/135 [==============================] - 10s 74ms/step - loss: 0.0161 - acc: 0.9948 - val_loss: 0.0342 - val_acc: 0.9909

Epoch 00016: val_loss did not improve
Epoch 17/200
135/135 [==============================] - 10s 74ms/step - loss: 0.0147 - acc: 0.9963 - val_loss: 0.0279 - val_acc: 0.9932

Epoch 00017: val_loss improved from 0.03023 to 0.02788, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice4_Testing_MMMU35710.h5
Epoch 18/200
135/135 [==============================] - 10s 74ms/step - loss: 0.0076 - acc: 0.9978 - val_loss: 0.0287 - val_acc: 0.9886

Epoch 00018: val_loss did not improve
Epoch 19/200
135/135 [==============================] - 10s 74ms/step - loss: 0.0096 - acc: 0.9970 - val_loss: 0.0240 - val_acc: 0.9932

Epoch 00019: val_loss improved from 0.02788 to 0.02403, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice4_Testing_MMMU35710.h5
Epoch 20/200
135/135 [==============================] - 10s 74ms/step - 

135/135 [==============================] - 10s 75ms/step - loss: 0.3818 - acc: 0.8607 - val_loss: 1.8212 - val_acc: 0.4545

Epoch 00002: val_loss improved from 3.16528 to 1.82116, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice4_Testing_MMMU35414.h5
Epoch 3/200
135/135 [==============================] - 10s 75ms/step - loss: 0.3422 - acc: 0.8658 - val_loss: 0.6661 - val_acc: 0.8114

Epoch 00003: val_loss improved from 1.82116 to 0.66606, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice4_Testing_MMMU35414.h5
Epoch 4/200
135/135 [==============================] - 10s 75ms/step - loss: 0.2350 - acc: 0.9081 - val_loss: 0.6325 - val_acc: 0.7136

Epoch 00004: val_loss improved from 0.66606 to 0.63250, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice4_Testing_MMMU35414.h5
Epoch 5/200
135/135 [==============================] - 10s 75ms/step - loss: 0.2293 - acc: 0.9133 - val_loss: 0.3136 - val_acc: 0.9091

Epoch 00005:

...

Epoch 26/200
146/146 [==============================] - 11s 73ms/step - loss: 0.0088 - acc: 0.9979 - val_loss: 0.0219 - val_acc: 0.9935

Epoch 00026: val_loss did not improve
Epoch 27/200
146/146 [==============================] - 11s 73ms/step - loss: 0.0066 - acc: 0.9986 - val_loss: 0.0178 - val_acc: 0.9935

Epoch 00027: val_loss did not improve
Epoch 28/200
146/146 [==============================] - 11s 73ms/step - loss: 0.0059 - acc: 0.9986 - val_loss: 0.0192 - val_acc: 0.9935

Epoch 00028: val_loss did not improve
Epoch 29/200
146/146 [==============================] - 11s 73ms/step - loss: 0.0090 - acc: 0.9979 - val_loss: 0.0169 - val_acc: 0.9935

Epoch 00029: val_loss did not improve
Epoch 30/200
146/146 [==============================] - 11s 73ms/step - loss: 0.0059 - acc: 0.9986 - val_loss: 0.0212 - val_acc: 0.9935

Epoch 00030: val_loss did not improve
Epoch 31/200
146/146 [==============================] - 11s 73ms/step - loss: 0.0098 - acc: 0.9979 - val_loss: 0.0158 - val_a


Epoch 00005: val_loss improved from 0.68080 to 0.47177, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice8_Testing_MMMU35414.h5
Epoch 6/200
135/135 [==============================] - 10s 77ms/step - loss: 0.1440 - acc: 0.9474 - val_loss: 2.9545 - val_acc: 0.7909

Epoch 00006: val_loss did not improve
Epoch 7/200
135/135 [==============================] - 10s 77ms/step - loss: 0.1390 - acc: 0.9459 - val_loss: 0.0650 - val_acc: 0.9795

Epoch 00007: val_loss improved from 0.47177 to 0.06503, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice8_Testing_MMMU35414.h5
Epoch 8/200
135/135 [==============================] - 10s 76ms/step - loss: 0.1487 - acc: 0.9511 - val_loss: 4.1274 - val_acc: 0.4568

Epoch 00008: val_loss did not improve
Epoch 9/200
135/135 [==============================] - 10s 77ms/step - loss: 0.1045 - acc: 0.9547 - val_loss: 3.1611 - val_acc: 0.4295

Epoch 00009: val_loss did not improve
Epoch 10/200
135/135 [===============

Epoch 5/200
156/156 [==============================] - 11s 71ms/step - loss: 0.3822 - acc: 0.8410 - val_loss: 0.6775 - val_acc: 0.7333

Epoch 00005: val_loss did not improve
Epoch 6/200
156/156 [==============================] - 11s 71ms/step - loss: 0.3516 - acc: 0.8519 - val_loss: 0.3758 - val_acc: 0.8944

Epoch 00006: val_loss did not improve
Epoch 7/200
156/156 [==============================] - 11s 71ms/step - loss: 0.3465 - acc: 0.8462 - val_loss: 0.2598 - val_acc: 0.9111

Epoch 00007: val_loss improved from 0.28893 to 0.25982, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice9_Testing_MMMU36727.h5
Epoch 8/200
156/156 [==============================] - 11s 71ms/step - loss: 0.3289 - acc: 0.8635 - val_loss: 0.5212 - val_acc: 0.7556

Epoch 00008: val_loss did not improve
Epoch 9/200
156/156 [==============================] - 11s 71ms/step - loss: 0.3185 - acc: 0.8705 - val_loss: 0.4232 - val_acc: 0.8111

Epoch 00009: val_loss did not improve
Epoch 10/200
156


Epoch 00015: val_loss did not improve
Epoch 16/200
146/146 [==============================] - 11s 73ms/step - loss: 0.2135 - acc: 0.9192 - val_loss: 0.7877 - val_acc: 0.8262

Epoch 00016: val_loss did not improve
Epoch 17/200
146/146 [==============================] - 11s 73ms/step - loss: 0.1838 - acc: 0.9260 - val_loss: 0.4529 - val_acc: 0.8885

Epoch 00017: val_loss did not improve
Epoch 18/200
146/146 [==============================] - 11s 74ms/step - loss: 0.1518 - acc: 0.9507 - val_loss: 0.2014 - val_acc: 0.9344

Epoch 00018: val_loss improved from 0.31869 to 0.20145, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice9_Testing_MMMU36365.h5
Epoch 19/200
146/146 [==============================] - 11s 74ms/step - loss: 0.0829 - acc: 0.9726 - val_loss: 0.1231 - val_acc: 0.9548

Epoch 00019: val_loss improved from 0.20145 to 0.12306, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice9_Testing_MMMU36365.h5
Epoch 20/200
146/146 [===========

135/135 [==============================] - 10s 78ms/step - loss: 0.1824 - acc: 0.9289 - val_loss: 0.1937 - val_acc: 0.9091

Epoch 00008: val_loss improved from 0.29369 to 0.19369, saving model to WEIGHTS_LeftlLung_Including_W16_W20/InceptionResNet_Slice9_Testing_MMMU35446.h5
Epoch 9/200
135/135 [==============================] - 11s 78ms/step - loss: 0.1607 - acc: 0.9393 - val_loss: 0.6334 - val_acc: 0.7386

Epoch 00009: val_loss did not improve
Epoch 10/200
135/135 [==============================] - 10s 78ms/step - loss: 0.1391 - acc: 0.9435 - val_loss: 0.8533 - val_acc: 0.7432

Epoch 00010: val_loss did not improve
Epoch 11/200
135/135 [==============================] - 11s 78ms/step - loss: 0.1330 - acc: 0.9480 - val_loss: 0.2401 - val_acc: 0.9045

Epoch 00011: val_loss did not improve
Epoch 12/200
135/135 [==============================] - 11s 78ms/step - loss: 0.1449 - acc: 0.9488 - val_loss: 0.3451 - val_acc: 0.8841

Epoch 00012: val_loss did not improve
Epoch 13/200
135/135 [===

In [13]:
def dict_counter(x):
    return dict(Counter(x))

def countErrors(pred, truth, week):
    return dict_counter([w for p, t, w in zip(pred, truth, week) if p != t])

df['week_errors'] = df.apply(lambda x : countErrors(x['prediction'], x['truth'], x['actual_weeks']), axis=1)
df['week_counts'] = df.apply(lambda x : dict_counter(x['actual_weeks']), axis =1)

In [14]:
df.to_csv(outputCsv, index=False)
df.to_pickle(outputPkl)
df

,slice,monkey,model_name,prediction,truth,actual_weeks,precision,recall,f1,week_errors,week_counts
0,4,MMMU36727,InceptionResNet_Slice4_Testing_MMMU36727,"[tb, tb, tb, notb, tb, tb, tb, tb, tb, notb, t...","[tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, t...","[W12, W12, W4, W8, W8, W12, W8, W8, W4, W4, W4...",0.500000,0.441799,0.469101,"{'W8': 13, 'W4': 9}","{'W12': 63, 'W4': 63, 'W8': 63}"
1,4,MMMU36365,InceptionResNet_Slice4_Testing_MMMU36365,"[tb, tb, tb, tb, notb, tb, notb, tb, tb, notb,...","[notb, tb, tb, tb, notb, tb, notb, tb, tb, not...","[W0, W12, W8, W8, W0, W8, W2, W4, W8, W2, W4, ...",0.777408,0.759259,0.768227,"{'W0': 28, 'W4': 12, 'W2': 16, 'W8': 1, 'W12':...","{'W0': 63, 'W12': 63, 'W8': 63, 'W2': 63, 'W4'..."
2,4,MMMU35710,InceptionResNet_Slice4_Testing_MMMU35710,"[tb, tb, tb, notb, notb, tb, tb, notb, tb, not...","[tb, tb, tb, notb, tb, tb, tb, tb, tb, tb, tb,...","[W12, W20, W12, W0, W4, W8, W4, W12, W16, W4, ...",0.680489,0.653968,0.666965,"{'W4': 26, 'W12': 14, 'W2': 62, 'W0': 8, 'W16'...","{'W12': 63, 'W20': 63, 'W0': 63, 'W4': 63, 'W8..."
3,4,MMMU35603,InceptionResNet_Slice4_Testing_MMMU35603,"[tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, t...","[tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, t...","[W16, W12, W12, W4, W20, W12, W20, W4, W4, W4,...",0.500000,0.484127,0.491935,"{'W20': 6, 'W16': 3, 'W12': 1}","{'W16': 63, 'W12': 63, 'W4': 63, 'W20': 63, 'W..."
4,4,MMMU35446,InceptionResNet_Slice4_Testing_MMMU35446,"[tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, t...","[tb, tb, tb, notb, tb, tb, tb, tb, tb, tb, tb,...","[W20, W12, W4, W2, W16, W4, W16, W12, W8, W8, ...",0.580362,0.507886,0.541710,"{'W2': 63, 'W0': 59, 'W4': 2, 'W12': 2, 'W16': 1}","{'W20': 62, 'W12': 63, 'W4': 63, 'W2': 63, 'W1..."
5,4,MMMU35414,InceptionResNet_Slice4_Testing_MMMU35414,"[tb, tb, tb, tb, tb, notb, tb, tb, tb, tb, tb,...","[tb, notb, tb, tb, tb, notb, tb, notb, tb, tb,...","[W16, W2, W4, W8, W8, W0, W8, W2, W20, W4, W8,...",0.627817,0.624000,0.625903,"{'W2': 62, 'W12': 49, 'W16': 5, 'W0': 7, 'W4': 9}","{'W16': 63, 'W2': 63, 'W4': 63, 'W8': 63, 'W0'..."
6,5,MMMU36727,InceptionResNet_Slice5_Testing_MMMU36727,"[tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, t...","[tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, t...","[W12, W8, W4, W4, W12, W8, W4, W8, W12, W12, W...",1.000000,1.000000,1.000000,{},"{'W12': 63, 'W8': 63, 'W4': 63}"
7,5,MMMU36365,InceptionResNet_Slice5_Testing_MMMU36365,"[notb, tb, notb, tb, notb, notb, tb, tb, tb, t...","[notb, tb, notb, notb, notb, notb, tb, tb, tb,...","[W0, W8, W0, W2, W0, W0, W8, W12, W4, W12, W4,...",0.866398,0.805471,0.834825,"{'W2': 37, 'W4': 6, 'W0': 8}","{'W0': 63, 'W8': 63, 'W2': 63, 'W12': 63, 'W4'..."
8,5,MMMU35710,InceptionResNet_Slice5_Testing_MMMU35710,"[tb, tb, tb, tb, tb, tb, tb, tb, notb, tb, tb,...","[tb, tb, tb, tb, tb, notb, tb, tb, notb, tb, t...","[W4, W20, W16, W12, W8, W2, W20, W16, W0, W16,...",0.825826,0.748413,0.785216,"{'W2': 43, 'W0': 14, 'W8': 9, 'W12': 7}","{'W4': 63, 'W20': 63, 'W16': 63, 'W12': 63, 'W..."
9,5,MMMU35603,InceptionResNet_Slice5_Testing_MMMU35603,"[tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, t...","[tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, tb, t...","[W12, W4, W16, W8, W12, W20, W8, W4, W4, W20, ...",0.500000,0.449206,0.473244,"{'W12': 16, 'W8': 14, 'W16': 1, 'W20': 1}","{'W12': 63, 'W4': 63, 'W16': 63, 'W8': 63, 'W2..."
